# Anatomy of a Quantizer

## What's in a Quantizer? 

In a broad sense, a quantizer is anything that implements a quantization technique, and the flexibility of Brevitas means that there are different ways to do so.  
However, to keep our terminology straight, we refer to a quantizer as a specific kind of way to implement quantization, the one preferred and adopted by default. That is, a quantizer is a subclass of a `brevitas.inject.ExtendedInjector` that carries a `tensor_quant` attribute.  
We have seen in previous tutorials quantizers being imported from `brevitas.quant` and passed on to quantized layers. We can easily very what we just said on one of them:

In [1]:
from brevitas.inject import ExtendedInjector
from brevitas.quant.scaled_int import Int8ActPerTensorFloat 

issubclass(Int8ActPerTensorFloat, ExtendedInjector)

No CUDA runtime is found, using CUDA_HOME='C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v10.2'


True

In [2]:
Int8ActPerTensorFloat.tensor_quant

RescalingIntQuant(
  (int_quant): IntQuant(
    (float_to_int_impl): RoundSte()
    (tensor_clamp_impl): TensorClamp()
    (delay_wrapper): DelayWrapper(
      (delay_impl): _NoDelay()
    )
  )
  (scaling_impl): ParameterFromRuntimeStatsScaling(
    (stats_input_view_shape_impl): OverTensorView()
    (stats): _Stats(
      (stats_impl): AbsPercentile()
    )
    (restrict_clamp_scaling): _RestrictClampValue(
      (clamp_min_ste): Identity()
      (restrict_value_impl): FloatRestrictValue()
    )
    (restrict_inplace_preprocess): Identity()
  )
  (int_scaling_impl): IntScaling()
  (zero_point_impl): ZeroZeroPoint(
    (zero_point): StatelessBuffer()
  )
  (msb_clamp_bit_width_impl): BitWidthConst(
    (bit_width): StatelessBuffer()
  )
)

Note how we said *subclass* and not *instance*. To understand why that's the case, we have to understand what an `ExtendedInjector` is and why it's used in the first place.

## Quantization with auto-wiring Dependency Injection

Pytorch has exploded in popularity thanks to its straightforward numpy-like *define-by-run* execution model. However, when it comes to applying quantization, this style of programming poses a problem.  

Many quantization methods depend on making decisions based on the (in Pytorch terms) `state_dict` of the original floating-point model to finetune with quantization. However, when we instantiate a model in Pytorch we can't know on the spot if a state_dict is going to be loaded a few lines of code later or not. Yet, because Pytorch is define-by-run, we need our model to work consistently both before and after a `state_dict` is possibly loaded. In a traditional scenario that wouldn't pose a problem. However, with quantization in the loop, the way a quantizer is defined might change before and after a pretrained `state_dict` is loaded.

That means that we need a way to define our quantized model such that it can react appropriately in case the `state_dict` changes. In a Python-only world that wouldn't be too hard. However, in order to mitigate the performance impact of quantization-aware training, Brevitas makes extended use of Pytorch's JIT compiler for a custom subset of Python, TorchScript. That means that in most scenarios, when a `state_dict` is loaded, we need to recompile parts of the model. Because compilation in general is a lossy process, a TorchScript component cannot simply re-compile itself based on new input information. 

We need then a way to *declare* a quantization method such that it can be re-initialized and JIT compiled any time the `state_dict` changes. Because we want to support arbitrarly-complex user-defined quantization algorithms, this method has to be generic, i.e. it cannot depend on the specifics of the quantization algorithm implemented.

Implementing a quantizer with an `ExtendedInjector` is a way to do so. Specifically, an `ExtendedInjector` extends an `Injector` from an older version (0.2.1) of the excellent dependency-injection *[dependencies](https://github.com/proofit404/dependencies)* library with support for a couple of extra features that are specific to Brevitas' needs.

An `Injector` (and an `ExtendedInjector`) allows to take what might be a very complicated graph of interwined objects and turns it into a flat list of variables that are capable of auto-assembly by matching variable names to arguments names. This technique typically goes under the name of auto-wiring dependency injection. 

In the context of Brevitas, the goal is gather all the modules and hyperparameters that contribute to a quantization implementation such that they can be re-assembled automatically on demand. What comes out of this process is a `tensor_quant` object.

## A Practical Example: Binary Quantization

To make things practical, let's look at how we can implement a simple variant of binary quantization. All the components typically used to implement quantization can be found under `brevitas.core`. As mentioned before, Brevitas makes heavy use of TorchScript. In particular, all the components found under `brevitas.core` are implemented as `ScriptModule` that can be assembled together.
The core `ScriptModule` that implements binarization can be found under `brevitas.core.quant`: 

In [3]:
import inspect
from IPython.display import Markdown, display

def pretty_print_source(source):
    display(Markdown('```python\n' + source + '\n```'))

In [4]:
from brevitas.core.quant import BinaryQuant

source = inspect.getsource(BinaryQuant)  
pretty_print_source(source)

```python
class BinaryQuant(brevitas.jit.ScriptModule):
    """
    ScriptModule that implements scaled uniform binary quantization of an input tensor.
    Quantization is performed with :func:`~brevitas.function.ops_ste.binary_sign_ste`.

    Args:
        scaling_impl (Module): Module that returns a scale factor.
        quant_delay_steps (int): Number of training steps to delay quantization for. Default: 0

    Returns:
        Tuple[Tensor, Tensor, Tensor, Tensor]: Quantized output in de-quantized format, scale,
            zero-point, bit_width.

    Examples:
        >>> from brevitas.core.scaling import ConstScaling
        >>> binary_quant = BinaryQuant(ConstScaling(0.1))
        >>> inp = torch.Tensor([0.04, -0.6, 3.3])
        >>> out, scale, zero_point, bit_width = binary_quant(inp)
        >>> out
        tensor([ 0.1000, -0.1000,  0.1000])
        >>> scale
        tensor(0.1000)
        >>> zero_point
        tensor(0.)
        >>> bit_width
        tensor(1.)

    Note:
        Maps to quant_type == QuantType.BINARY == 'BINARY' == 'binary' when applied to weights
         in higher-level APIs.

    Note:
        Set env variable BREVITAS_JIT=1 to enable TorchScript compilation of this module.
    """

    def __init__(self, scaling_impl: Module, quant_delay_steps: int = 0):
        super(BinaryQuant, self).__init__()
        self.scaling_impl = scaling_impl
        self.bit_width = BitWidthConst(1)
        self.zero_point = StatelessBuffer(torch.tensor(0.0))
        self.delay_wrapper = DelayWrapper(quant_delay_steps)

    @brevitas.jit.script_method
    def forward(self, x: Tensor) -> Tuple[Tensor, Tensor, Tensor, Tensor]:
        scale = self.scaling_impl(x)
        y = binary_sign_ste(x) * scale
        y = self.delay_wrapper(x, y)
        return y, scale, self.zero_point(), self.bit_width()

```

The implementation is quite simple. Apart from `quant_delay_steps`, which allows to delay quantization by a certain number of training steps (*default = 0*), the only other argument that BinaryQuant accepts is an implementation to compute the scale factor. `bit_width` is fixed to 1 and `zero-point` is fixed to 0.

We pick as scale factor implementation a `ScriptModule` called `ParameterScaling`, which implements a learned parameter with user-defined initialization. It can be found under `brevitas.core.scaling`:

In [5]:
from brevitas.core.scaling import ParameterScaling

### Manual Binary Quantization

As a first step, we simply instantiate `BinaryQuant` with `ParameterScaling` using `scaling_init` equal *0.1* and we call it on a random floating-point input tensor:

In [6]:
import torch

manual_tensor_quant = BinaryQuant(scaling_impl=ParameterScaling(scaling_init=0.1))
manual_tensor_quant(torch.randn(4, 4))

(tensor([[-0.1000,  0.1000, -0.1000, -0.1000],
         [ 0.1000,  0.1000,  0.1000,  0.1000],
         [ 0.1000,  0.1000, -0.1000,  0.1000],
         [ 0.1000, -0.1000,  0.1000,  0.1000]], grad_fn=<MulBackward0>),
 tensor(0.1000, grad_fn=<AbsBinarySignGradFnBackward>),
 tensor(0.),
 tensor(1.))

Nothing too surprising here, as expected the tensor is binarized with the scale factor we defined. Note however how `manual_tensor_quant` is returning a `tuple` and not a `QuantTensor`. This is because support for custom data structures in TorchScript is still quite limited, so `QuantTensor` are allocated only in Python-world abstractions.

### Binary Quantization with an ExtendedInjector

Let's now declare `tensor_quant` through an `ExtendedInjector`:

In [7]:
from brevitas.inject import ExtendedInjector

class MyBinaryQuantizer(ExtendedInjector):
    tensor_quant = BinaryQuant
    scaling_impl=ParameterScaling
    scaling_init=0.1

inj_tensor_quant = MyBinaryQuantizer.tensor_quant
inj_tensor_quant(torch.randn(4, 4))

(tensor([[ 0.1000,  0.1000, -0.1000,  0.1000],
         [ 0.1000,  0.1000,  0.1000, -0.1000],
         [-0.1000, -0.1000, -0.1000,  0.1000],
         [ 0.1000, -0.1000, -0.1000,  0.1000]], grad_fn=<MulBackward0>),
 tensor(0.1000, grad_fn=<AbsBinarySignGradFnBackward>),
 tensor(0.),
 tensor(1.))

Any time `MyBinaryQuantizer.tensor_quant` is called, a new instance of `BinaryQuant` is created. Note how the attributes of `MyBinaryQuantizer` are designed to match the name of the arguments of each other, except for `tensor_quant`, which is what we are interested in retrieving from the outside.

## Inheritance and Composition of Quantizers

The advantage of expressing a quantizer through a Python class also means that we can leverage both *inheritance* and *composition*. So for example we can inherit from `MyBinaryQuantizer` and override `scaling_init` with a new value:

In [8]:
class MyChildBinaryQuantizer(MyBinaryQuantizer):
    scaling_init=1.0
    
child_inj_tensor_quant = MyChildBinaryQuantizer.tensor_quant
child_inj_tensor_quant(torch.randn(4, 4))

(tensor([[ 1., -1.,  1., -1.],
         [ 1., -1., -1.,  1.],
         [ 1., -1.,  1.,  1.],
         [-1., -1.,  1., -1.]], grad_fn=<MulBackward0>),
 tensor(1., grad_fn=<AbsBinarySignGradFnBackward>),
 tensor(0.),
 tensor(1.))

Or we can leverage composition by assembling together various classes containing different pieces of a quantizer:

In [9]:
class MyBinaryImpl(ExtendedInjector):
    tensor_quant = BinaryQuant

class MyScalingImpl(ExtendedInjector):
    scaling_impl=ParameterScaling
    scaling_init=0.1
    
class MyComposedBinaryQuantizer(MyBinaryImpl, MyScalingImpl):
    pass

comp_inj_tensor_quant = MyComposedBinaryQuantizer.tensor_quant
comp_inj_tensor_quant(torch.randn(10, 10))

(tensor([[-0.1000,  0.1000, -0.1000, -0.1000,  0.1000,  0.1000,  0.1000,  0.1000,
           0.1000,  0.1000],
         [-0.1000, -0.1000,  0.1000,  0.1000, -0.1000,  0.1000, -0.1000, -0.1000,
           0.1000,  0.1000],
         [ 0.1000, -0.1000, -0.1000,  0.1000,  0.1000, -0.1000,  0.1000, -0.1000,
          -0.1000, -0.1000],
         [-0.1000, -0.1000, -0.1000, -0.1000, -0.1000,  0.1000, -0.1000,  0.1000,
          -0.1000, -0.1000],
         [ 0.1000,  0.1000, -0.1000,  0.1000,  0.1000,  0.1000, -0.1000,  0.1000,
           0.1000, -0.1000],
         [ 0.1000, -0.1000, -0.1000, -0.1000,  0.1000, -0.1000, -0.1000, -0.1000,
           0.1000,  0.1000],
         [ 0.1000, -0.1000, -0.1000,  0.1000,  0.1000, -0.1000,  0.1000,  0.1000,
           0.1000,  0.1000],
         [-0.1000, -0.1000, -0.1000, -0.1000,  0.1000,  0.1000, -0.1000, -0.1000,
           0.1000,  0.1000],
         [-0.1000, -0.1000,  0.1000, -0.1000, -0.1000, -0.1000,  0.1000, -0.1000,
           0.1000,  0.1000],
 

## Passing a Custom Quantizer to QuantConv2d

As expected, we can pass the quantizer to a quantized layer such as QuantConv2d:

In [10]:
from brevitas.nn import QuantConv2d

binary_weight_quant_conv = QuantConv2d(3, 2, (3,3), weight_quant=MyBinaryQuantizer)
quant_weight = binary_weight_quant_conv.quant_weight()
quant_weight

QuantTensor(value=tensor([[[[ 0.1000, -0.1000,  0.1000],
          [ 0.1000, -0.1000,  0.1000],
          [-0.1000, -0.1000,  0.1000]],

         [[-0.1000, -0.1000,  0.1000],
          [-0.1000,  0.1000, -0.1000],
          [ 0.1000,  0.1000, -0.1000]],

         [[ 0.1000,  0.1000, -0.1000],
          [-0.1000, -0.1000, -0.1000],
          [-0.1000,  0.1000, -0.1000]]],


        [[[ 0.1000,  0.1000,  0.1000],
          [-0.1000,  0.1000,  0.1000],
          [-0.1000,  0.1000,  0.1000]],

         [[-0.1000, -0.1000, -0.1000],
          [ 0.1000, -0.1000, -0.1000],
          [-0.1000,  0.1000,  0.1000]],

         [[ 0.1000, -0.1000, -0.1000],
          [-0.1000,  0.1000,  0.1000],
          [ 0.1000,  0.1000,  0.1000]]]], grad_fn=<MulBackward0>), scale=tensor(0.1000, grad_fn=<AbsBinarySignGradFnBackward>), zero_point=tensor(0.), bit_width=tensor(1.), signed=None, training=True)

Note however how the `QuantTensor` is not properly formed, as the `signed` attribute is `None`. This means that `quant_weight` is not considered valid, as the affine quantization invariant cannot be computed:

In [11]:
quant_weight.is_valid

False

`signed` is one of those attributes that in the case of binary quantization has to be explicitly defined by the user. We can do so by simply setting it in the quantizer:

In [12]:
class MySignedBinaryQuantizer(MyBinaryQuantizer):
    signed = True
    
binary_weight_quant_conv = QuantConv2d(3, 2, (3,3), weight_quant=MySignedBinaryQuantizer)
signed_quant_weight = binary_weight_quant_conv.quant_weight()
signed_quant_weight

QuantTensor(value=tensor([[[[-0.1000, -0.1000, -0.1000],
          [ 0.1000,  0.1000, -0.1000],
          [ 0.1000, -0.1000,  0.1000]],

         [[-0.1000,  0.1000,  0.1000],
          [-0.1000, -0.1000,  0.1000],
          [ 0.1000, -0.1000,  0.1000]],

         [[-0.1000,  0.1000, -0.1000],
          [-0.1000,  0.1000,  0.1000],
          [-0.1000,  0.1000, -0.1000]]],


        [[[-0.1000, -0.1000, -0.1000],
          [-0.1000,  0.1000, -0.1000],
          [ 0.1000, -0.1000, -0.1000]],

         [[-0.1000, -0.1000,  0.1000],
          [-0.1000,  0.1000,  0.1000],
          [-0.1000, -0.1000, -0.1000]],

         [[-0.1000,  0.1000,  0.1000],
          [-0.1000,  0.1000,  0.1000],
          [-0.1000, -0.1000, -0.1000]]]], grad_fn=<MulBackward0>), scale=tensor(0.1000, grad_fn=<AbsBinarySignGradFnBackward>), zero_point=tensor(0.), bit_width=tensor(1.), signed=True, training=True)

In [13]:
signed_quant_weight.is_valid

True

And now the quant weights are valid.

When we want to add or override an single attribute of a quantizer passed to a layer, defining a whole new quantizer can be too verbose. There is a simpler syntax to achieve the same goal. Let's say we want to have `scaling_init = 0.0001`. We can simply do the following:

In [14]:
small_scale_quant_conv = QuantConv2d(3, 2, (3,3), weight_quant=MySignedBinaryQuantizer, weight_scaling_init=0.001)
small_scale_quant_conv.quant_weight()

QuantTensor(value=tensor([[[[-0.0010,  0.0010, -0.0010],
          [-0.0010, -0.0010, -0.0010],
          [-0.0010,  0.0010, -0.0010]],

         [[ 0.0010, -0.0010, -0.0010],
          [-0.0010,  0.0010, -0.0010],
          [-0.0010,  0.0010, -0.0010]],

         [[ 0.0010, -0.0010, -0.0010],
          [-0.0010, -0.0010,  0.0010],
          [ 0.0010,  0.0010,  0.0010]]],


        [[[-0.0010,  0.0010,  0.0010],
          [ 0.0010,  0.0010, -0.0010],
          [-0.0010, -0.0010,  0.0010]],

         [[-0.0010, -0.0010,  0.0010],
          [-0.0010,  0.0010, -0.0010],
          [ 0.0010,  0.0010, -0.0010]],

         [[-0.0010, -0.0010,  0.0010],
          [ 0.0010,  0.0010,  0.0010],
          [-0.0010, -0.0010, -0.0010]]]], grad_fn=<MulBackward0>), scale=tensor(0.0010, grad_fn=<AbsBinarySignGradFnBackward>), zero_point=tensor(0.), bit_width=tensor(1.), signed=True, training=True)

What we did was to take the name of the attribute `scaling_init`, add the prefix `weight_`, and pass it as a keyword argument to `QuantConv2d`. What happens in the background is that the keyword arguments prefixed with `weight_` are set as attributes of `weight_quant`, possibly overriding any pre-existing value. The same principle applies to `input_`, `output_` and `bias_`. In case the corrisponding quantizer is `None`, a new *empty* `ExtendedInjector` is internally allocated and attributes are passed to that. So for example we can define `output_quant` (which by default is None) completely *inline* without defining an explicit standalone quantizer: 

In [15]:
inline_output_quant = QuantConv2d(
    3, 2, (3,3), 
    weight_quant=MySignedBinaryQuantizer,
    output_tensor_quant = BinaryQuant,
    output_scaling_impl=ParameterScaling,
    output_scaling_init=1.0)

inline_output_quant(torch.randn(1, 3, 5, 5))

tensor([[[[ 1., -1.,  1.],
          [-1.,  1., -1.],
          [ 1.,  1., -1.]],

         [[ 1., -1.,  1.],
          [-1.,  1.,  1.],
          [ 1.,  1.,  1.]]]], grad_fn=<MulBackward0>)

This is the reason why, as it was mentioned in the first tutorial, quantized layers can accept arbitrary keyword arguments. It's really just a way to support different styles of syntax when defining a quantizer.

## Passing a custom quantizer to QuantReLU

We can do the same thing with quantized activations:

In [16]:
from brevitas.nn import QuantReLU

binary_relu = QuantReLU(act_quant=MySignedBinaryQuantizer)
binary_relu(torch.randn(4, 4))

tensor([[0.1000, 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000],
        [0.1000, 0.1000, 0.1000, 0.1000]], grad_fn=<MulBackward0>)

So there isn't really much difference between a quantizer for weights and a quantizer for activations. The difference with activations is that a prefix is not required when passing keyword arguments:

In [17]:
small_scale_binary_relu = QuantReLU(act_quant=MySignedBinaryQuantizer, scaling_init=0.001)
small_scale_binary_relu(torch.randn(4, 4))

tensor([[0.0010, 0.0010, 0.0010, 0.0010],
        [0.0010, 0.0010, 0.0010, 0.0010],
        [0.0010, 0.0010, 0.0010, 0.0010],
        [0.0010, 0.0010, 0.0010, 0.0010]], grad_fn=<MulBackward0>)

## A Custom Quantizer initialized with Weight Statistics

So far we have seen use-cases where an `ExtendedInjector` provides at best a different kind of syntax to define a quantizer, without any particular other advantage. Let's now make things a bit more complicated to show the sort of situations where it really shines. 

Let's say we want to define a binary weight quantizer where `scaling_impl` is still `ParameterScaling`. However, instead of being user-defined, we want `scaling_init` to be the maximum value found in the weight tensor of the quantized layer.
To support this sort of use cases where the quantizer depends on the layer, a quantized layer automatically passes itself to all its quantizers under the name of `module`. With some machinery then, we can achieve our goal:

In [18]:
from brevitas.inject import value

class ParamFromMaxWeightQuantizer(MySignedBinaryQuantizer):
    
    @value
    def scaling_init(module):
        return module.weight.abs().max()

Note how we are leveraging the `@value` *decorator* to define a function that is executed at *dependency-injection (DI) time*. This kind of behaviour is similar to defining a @property instead of an attribute, with the difference that a @value function can depend on other attributes of the Injector, which are automatically passed in as arguments of the function during DI. 

Let's now pass the quantizer to a QuantConv2d and retrieve its quantized weights:

In [19]:
param_from_max_quant_conv = QuantConv2d(3, 2, (3, 3), weight_quant=ParamFromMaxWeightQuantizer)
param_from_max_quant_conv.quant_weight()

QuantTensor(value=tensor([[[[-0.1848, -0.1848, -0.1848],
          [ 0.1848,  0.1848, -0.1848],
          [ 0.1848,  0.1848,  0.1848]],

         [[-0.1848,  0.1848,  0.1848],
          [ 0.1848, -0.1848, -0.1848],
          [ 0.1848, -0.1848,  0.1848]],

         [[-0.1848,  0.1848,  0.1848],
          [-0.1848,  0.1848,  0.1848],
          [-0.1848, -0.1848, -0.1848]]],


        [[[ 0.1848, -0.1848,  0.1848],
          [-0.1848, -0.1848,  0.1848],
          [ 0.1848, -0.1848,  0.1848]],

         [[ 0.1848, -0.1848,  0.1848],
          [ 0.1848, -0.1848, -0.1848],
          [-0.1848, -0.1848,  0.1848]],

         [[-0.1848, -0.1848, -0.1848],
          [ 0.1848,  0.1848, -0.1848],
          [ 0.1848, -0.1848, -0.1848]]]], grad_fn=<MulBackward0>), scale=tensor(0.1848, grad_fn=<AbsBinarySignGradFnBackward>), zero_point=tensor(0.), bit_width=tensor(1.), signed=True, training=True)

Indeed we can verify that `quant_weight_scale()` is equal to `weight.abs().max()`:

In [20]:
(param_from_max_quant_conv.quant_weight_scale() == param_from_max_quant_conv.weight.abs().max()).item()

True

Let's say now that we want to load a pretrained floating-point weight tensor on top of our quantized model. We simuate this scenario by defining a separate `nn.Conv2d` layer with the same weight shape:

In [21]:
from torch import nn

float_conv = nn.Conv2d(3, 2, (3, 3))
float_conv.weight.abs().max()

tensor(0.1913, grad_fn=<MaxBackward1>)

and then we load it on top of `param_from_max_quant_conv`:

In [22]:
param_from_max_quant_conv.load_state_dict(float_conv.state_dict())

RuntimeError: Error(s) in loading state_dict for QuantConv2d:
	Missing key(s) in state_dict: "weight_quant.tensor_quant.scaling_impl.value". 

Ouch, we get an error. This is because `ParameterScaling` contains a learned parameter, and Pytorch expects all learned parameters of a model to be contained in a state dict.  
We can work around the issue by either setting an appropriate config flag in Brevitas, or by passing `strict=False` to load_state_dict. We go with the first way as setting `strict=False` is too forgiving to other kind of problems:

In [23]:
from brevitas import config
config.IGNORE_MISSING_KEYS = True

param_from_max_quant_conv.load_state_dict(float_conv.state_dict())

<All keys matched successfully>

Note that we could have also achieve the same goal by setting the *env variable* `BREVITAS_IGNORE_MISSING_KEYS=1`.

And now we can look at the quantized weights again:

In [24]:
param_from_max_quant_conv.quant_weight()

QuantTensor(value=tensor([[[[ 0.1913,  0.1913, -0.1913],
          [-0.1913, -0.1913,  0.1913],
          [-0.1913, -0.1913,  0.1913]],

         [[-0.1913, -0.1913, -0.1913],
          [ 0.1913,  0.1913,  0.1913],
          [-0.1913,  0.1913,  0.1913]],

         [[ 0.1913,  0.1913,  0.1913],
          [-0.1913,  0.1913, -0.1913],
          [-0.1913, -0.1913,  0.1913]]],


        [[[ 0.1913,  0.1913,  0.1913],
          [ 0.1913,  0.1913,  0.1913],
          [-0.1913, -0.1913, -0.1913]],

         [[-0.1913, -0.1913,  0.1913],
          [-0.1913,  0.1913, -0.1913],
          [-0.1913,  0.1913, -0.1913]],

         [[-0.1913, -0.1913, -0.1913],
          [ 0.1913, -0.1913, -0.1913],
          [-0.1913,  0.1913,  0.1913]]]], grad_fn=<MulBackward0>), scale=tensor(0.1913, grad_fn=<AbsBinarySignGradFnBackward>), zero_point=tensor(0.), bit_width=tensor(1.), signed=True, training=True)

As expected, the scale factor has been updated to the new `weight.abs().max()`. 

What happens internally is that after `load_state_dict` is called on the layer, `ParamFromMaxWeightQuantizer.tensor_quant` gets called again to re-initialize `BinaryQuant`, and in turn `ParameterScaling` is re-initialized with a new `scaling_init` value computed based on the updated `module.weight` tensor. This whole process wouldn't have been possible without an `ExtendedInjector` behind it. 

## Building a custom quantization API

Now let's make things even more complicated. We are going to look at a scenario that illustrates the differences between a standard `Injector` (implemented in the dependencies library) and our `ExtendedInjector` extension.  

Let's say we want to build two quantizers for respectively weights and activations and build a simple API on top of them. In particular, we want to be able to switch between `BinaryQuant` and `ClampedBinaryQuant` (a variant of binary quantization with clamping typically used in activation quantization), and we want to optionally perform *per-channel scaling*. 
We can go as follows:

In [25]:
from brevitas.core.quant import ClampedBinaryQuant
from brevitas.inject import this


class CommonQuantizer(ExtendedInjector):
    scaling_impl = ParameterScaling
    signed=True
    
    @value
    def tensor_quant(is_clamped):
        # returning a class to auto-wire from a value function
        # wouldn't be allowed in a standard Injector
        if is_clamped:
            return ClampedBinaryQuant
        else:
            return BinaryQuant
    
    @value
    def scaling_shape(scaling_per_output_channel):
        if scaling_per_output_channel:
            # returning this.something from a value function 
            # wouldn't be allowed in a standard Injector
            return this.per_channel_broadcastable_shape
        else:
            return ()
        
        
class AdvancedWeightQuantizer(CommonQuantizer):
        
    @value
    def per_channel_broadcastable_shape(module):
        return (module.weight.shape[0], 1, 1, 1)
    
    @value
    def scaling_init(module, scaling_per_output_channel):
        if scaling_per_output_channel:
            num_ch = module.weight.shape[0]
            return module.weight.abs().view(num_ch, -1).max(dim=1)[0].view(-1, 1, 1, 1)
        else:
            return module.weight.abs().max()
        
        
class AdvancedActQuantizer(CommonQuantizer):
    scaling_init = 0.01

There are a bunch of things going on here to unpack.  

The first one is that a `@value` function can return a class to auto-wire and inject, as seen in the definition of `tensor_quant`. This wouldn't normally be possible with a standard `Injector`, but it's possible with an `ExtendedInjector`. This way we can switch between different implementations of `tensor_quant` with a boolean flag.  

The second one is the special object `this`. `this` is already present in the *dependencies* library, and it's used as a way to retrieve attributes of the quantizer from within the quantizer itself. However, normally it wouldn't be possible to return a reference to `this` from a `@value` function. Again this is something that only a `ExtendedInjector` supports, and it allows to chain different attributes in a way such that the chained values are computed only when necessary. 

Let's see the quantizers applied to a layer:

In [26]:
per_channel_quant_conv = QuantConv2d(
    3, 2, (3, 3), 
    weight_quant=AdvancedWeightQuantizer, 
    weight_is_clamped=False, 
    weight_scaling_per_output_channel=True)
per_channel_quant_conv.quant_weight()

QuantTensor(value=tensor([[[[ 0.1718, -0.1718, -0.1718],
          [ 0.1718, -0.1718, -0.1718],
          [-0.1718,  0.1718, -0.1718]],

         [[-0.1718,  0.1718, -0.1718],
          [-0.1718, -0.1718, -0.1718],
          [ 0.1718,  0.1718, -0.1718]],

         [[-0.1718,  0.1718,  0.1718],
          [-0.1718, -0.1718, -0.1718],
          [ 0.1718, -0.1718, -0.1718]]],


        [[[-0.1897, -0.1897,  0.1897],
          [ 0.1897,  0.1897,  0.1897],
          [-0.1897, -0.1897,  0.1897]],

         [[-0.1897, -0.1897,  0.1897],
          [ 0.1897,  0.1897, -0.1897],
          [-0.1897, -0.1897, -0.1897]],

         [[-0.1897, -0.1897, -0.1897],
          [ 0.1897, -0.1897,  0.1897],
          [-0.1897,  0.1897,  0.1897]]]], grad_fn=<MulBackward0>), scale=tensor([[[[0.1718]]],


        [[[0.1897]]]], grad_fn=<AbsBinarySignGradFnBackward>), zero_point=tensor(0.), bit_width=tensor(1.), signed=True, training=True)

As expected the weight scale is now a vector. Everything we said so far about quantizers still applies, so for example we can load the floating-point state dict we defined before and observe how it triggers an update of the weight scale:

In [27]:
per_channel_quant_conv.load_state_dict(float_conv.state_dict())
per_channel_quant_conv.quant_weight()

QuantTensor(value=tensor([[[[ 0.1913,  0.1913, -0.1913],
          [-0.1913, -0.1913,  0.1913],
          [-0.1913, -0.1913,  0.1913]],

         [[-0.1913, -0.1913, -0.1913],
          [ 0.1913,  0.1913,  0.1913],
          [-0.1913,  0.1913,  0.1913]],

         [[ 0.1913,  0.1913,  0.1913],
          [-0.1913,  0.1913, -0.1913],
          [-0.1913, -0.1913,  0.1913]]],


        [[[ 0.1889,  0.1889,  0.1889],
          [ 0.1889,  0.1889,  0.1889],
          [-0.1889, -0.1889, -0.1889]],

         [[-0.1889, -0.1889,  0.1889],
          [-0.1889,  0.1889, -0.1889],
          [-0.1889,  0.1889, -0.1889]],

         [[-0.1889, -0.1889, -0.1889],
          [ 0.1889, -0.1889, -0.1889],
          [-0.1889,  0.1889,  0.1889]]]], grad_fn=<MulBackward0>), scale=tensor([[[[0.1913]]],


        [[[0.1889]]]], grad_fn=<AbsBinarySignGradFnBackward>), zero_point=tensor(0.), bit_width=tensor(1.), signed=True, training=True)

In this case we have a per-channel quantizer, so the original floating-point weight tensor is now quantized per channel. 

Similarly, we can apply our custom activation quantizer to e.g. a `QuantIdentity` layer:

In [28]:
from brevitas.nn import QuantIdentity

quant_identity = QuantIdentity(
    act_quant=AdvancedActQuantizer, is_clamped=True, scaling_per_output_channel=False)
quant_identity(torch.randn(4, 4))

tensor([[-0.0100, -0.0100,  0.0100,  0.0100],
        [-0.0100,  0.0100, -0.0100,  0.0100],
        [ 0.0100,  0.0100,  0.0100,  0.0100],
        [-0.0100,  0.0100, -0.0100,  0.0100]], grad_fn=<MulBackward0>)

Note how `AdvancedActQuantizer` doesn't define a `per_channel_broadcastable_shape`, yet no errors are triggered. This is because `this.per_channel_broadcastable_shape` is required only when `scaling_per_output_channel` is `True`, while in this case `scaling_per_output_channel` is `False`. Let' try that then:

In [29]:
from brevitas.nn import QuantIdentity

quant_identity = QuantIdentity(
    act_quant=AdvancedActQuantizer, is_clamped=True, scaling_per_output_channel=True)

DependencyError: 'AdvancedActQuantizer' can not resolve attribute 'per_channel_broadcastable_shape'

As expected we get an error saying that the quantizer cannot resolve `per_channel_broadcastable_shape`. If we pass it in then we can get a per-channel quantizer:

In [30]:
quant_identity = QuantIdentity(
    act_quant=AdvancedActQuantizer, is_clamped=True, scaling_per_output_channel=True,
    per_channel_broadcastable_shape=(4, 1), return_quant_tensor=True)
quant_identity(torch.randn(4, 4))

QuantTensor(value=tensor([[ 0.0100, -0.0100,  0.0100,  0.0100],
        [-0.0100, -0.0100,  0.0100,  0.0100],
        [-0.0100, -0.0100,  0.0100,  0.0100],
        [ 0.0100, -0.0100,  0.0100,  0.0100]], grad_fn=<MulBackward0>), scale=tensor([[0.0100],
        [0.0100],
        [0.0100],
        [0.0100]], grad_fn=<AbsBinarySignGradFnBackward>), zero_point=tensor(0.), bit_width=tensor(1.), signed=True, training=True)

We have seen how powerful dependency injection is. In a way, it's even too expressive. For users that are not interesting in building completely custom quantizers it's a bit too much. In particular, it can be hard to make sense of how the various components available under `brevitas.core` can be assembled together according to be practices. 

In the next tutorial then we are going to take a look at an API implemented with the mechanisms we just saw that builds on top of the components available in `brevitas.core` and exposes easily switchable settings that follows best practices.